In [1]:
import numpy as np

from ffsim.testing import generate_norb_nelec
import ffsim

from ffsim import sample_slater

import math

In [2]:
generator = generate_norb_nelec(range(1, 5))

rng = np.random.default_rng(1234)
ffsim.random.random_unitary(5, seed=rng)


def _empirical_distribution(bts_matrix, norb, nelec):

    indices = np.zeros(bts_matrix.shape[0], dtype=int)
    for i, bts in enumerate(bts_matrix):
        string = np.array2string(bts, separator = '')[1:-1]
        index = ffsim.strings_to_indices([string], norb, nelec)[0]
        indices[i] = index

    
    unique_indices, counts = np.unique(indices, return_counts=True)

    if isinstance(nelec, tuple):

        probabilities = np.zeros(math.comb(norb, nelec[0])*math.comb(norb, nelec[1]))
    else:
        probabilities = np.zeros(math.comb(norb, nelec))


    probabilities[unique_indices] = counts

    probabilities /= np.sum(probabilities)

    return probabilities


def test_slater_sampler(norb, nelec):
    rng = np.random.default_rng(1234)

    n_samples = 8000

    n_a, n_b = nelec

    rotation_a = ffsim.random.random_unitary(norb, seed=rng)
    rotation_b = ffsim.random.random_unitary(norb, seed=rng)

    vecs_a = rotation_a[:, :n_a]
    vecs_b = rotation_b[:, :n_b]

    rdm_a = vecs_a @ np.conjugate(vecs_a.T)
    rdm_b = vecs_b @ np.conjugate(vecs_b.T)

    test_distribution = np.absolute(ffsim.slater_determinant(norb,
                                                (np.arange(n_a),np.arange(n_b)),
                                                (rotation_a, rotation_b))) ** 2
    
    

    samples = ffsim.sample_slater((rdm_a, rdm_b),
                                  n_samples//10, 10, 1, seed=rng)
    
    empirical_distribution = _empirical_distribution(samples, norb, nelec)


    assert np.sum(np.sqrt(test_distribution * empirical_distribution)) > 0.99

    print (np.sum(np.sqrt(test_distribution*  empirical_distribution)) )




print (test_slater_sampler(5, (3, 3)))

def test_slater_sampler_spinless(norb: int, nelec: int):
    """Test Slater determinant sampler."""
    rng = np.random.default_rng(1234)

    n_samples = 8000

    rotation = ffsim.random.random_unitary(norb, seed=rng)

    vecs = rotation[:, :nelec]

    rdm = vecs @ np.conjugate(vecs.T)

    test_distribution = np.absolute(ffsim.slater_determinant(norb,
                                                             (np.arange(nelec), np.arange(0)),
                                                             rotation)) ** 2

    samples = ffsim.sample_slater(rdm,
                                  n_samples//10, 10, 1, seed=rng)

    empirical_distribution = _empirical_distribution(samples, norb, nelec)

    assert np.sum(np.sqrt(test_distribution * empirical_distribution)) > 0.99

    print (np.sum(np.sqrt(test_distribution * empirical_distribution)))
    

/Users/javierrobledomoreno/miniconda3/envs/ffsim312/lib/python3.12/site-packages/numpy/linalg/linalg.py:2120: RuntimeWarning: divide by zero encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
/Users/javierrobledomoreno/miniconda3/envs/ffsim312/lib/python3.12/site-packages/numpy/linalg/linalg.py:2120: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)


0.9974870242393903
None


In [3]:
generator = ffsim.testing.generate_norb_nelec(range(1, 5))

for i in generator:
    print (i)
    test_slater_sampler(i[0],i[1])

(1, (0, 0))
1.0
(1, (0, 1))
1.0000000000000002
(1, (1, 0))
0.9999999999999999
(1, (1, 1))
1.0000000000000002
(2, (0, 0))
1.0
(2, (0, 1))
0.9999914159768536
(2, (0, 2))
1.0
(2, (1, 0))
0.9999989309998416
(2, (1, 1))
0.9999361515450114
(2, (1, 2))
0.9999970580864913
(2, (2, 0))
0.9999999999999998
(2, (2, 1))
0.9999771861816953
(2, (2, 2))
0.9999999999999998
(3, (0, 0))
1.0
(3, (0, 1))
0.9999293087005906
(3, (0, 2))
0.9999665969054823
(3, (0, 3))
0.9999999999999996
(3, (1, 0))
0.9999513322619554
(3, (1, 1))
0.9998813107495115
(3, (1, 2))
0.9999197781275226
(3, (1, 3))
0.9999045528199741
(3, (2, 0))
0.9999861000229096
(3, (2, 1))
0.9997934163446922
(3, (2, 2))
0.9999042604904955
(3, (2, 3))
0.9999402241674542
(3, (3, 0))
1.0000000000000004
(3, (3, 1))
0.9999965094574644
(3, (3, 2))
0.9999760107465889
(3, (3, 3))
1.0
(4, (0, 0))
1.0
(4, (0, 1))
0.9999952353664646
(4, (0, 2))
0.9997821023398057
(4, (0, 3))
0.9999738750314583
(4, (0, 4))
0.9999999999999997
(4, (1, 0))
0.9999917572666469
(4, (

In [4]:
generator = ffsim.testing.generate_norb_nocc(range(1, 5))

for i in generator:
    print (i)
    test_slater_sampler_spinless(i[0],i[1])

(1, 0)
1.0
(1, 1)
0.9999999999999999
(2, 0)
1.0
(2, 1)
0.9999973543324467
(2, 2)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [12]:
norb = 3

rotation = ffsim.random.random_unitary(norb, seed=rng)


ffsim.slater_determinant(norb,(np.arange(2), np.arange(0)),rotation)

array([ 0.65650439+0.08474979j, -0.3383185 -0.5506651j ,
        0.32514644-0.19597906j])

In [6]:
ffsim.

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [6]:
for sample in samples:
    print (sample)
    print (np.array2string(sample, separator=''))

[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 0 1]
[0000000101]
[0 0 0 0 0 0 0 1 0 1]
[0000000101]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 1 0 0 1 0]
[0000010010]
[0 0 0 0 0 0 0 0 1 1]
[0000000011]
[0 0 0 0 0 0 0 0 1 1]
[0000000011]
[0 0 0 0 0 1 0 0 0 1]
[0000010001]
[0 0 0 0 0 1 0 0 1 0]
[0000010010]
[0 0 0 0 0 1 0 0 1 0]
[0000010010]
[0 0 0 0 0 1 0 0 1 0]
[0000010010]
[0 0 0 0 0 1 0 0 1 0]
[0000010010]
[0 0 0 0 0 1 0 1 0 0]
[0000010100]
[0 0 0 0 0 0 1 1 0 0]
[0000001100]
[0 0 0 0 0 1 0 1 0 0]
[0000010100]
[0 0 0 0 0 0 0 1 0 1]
[0000000101]
[0 0 0 0 0 0 0 1 0 1]
[0000000101]
[0 0 0 0 0 1 0 1 0 0]
[0000010100]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0]
[0000000110]
[0 0 0 0 0 0 0 1 1 0